In [1]:
#basic imports
import pandas as pd
import numpy as np

#fitbit packages 
import fitbit
from python_fitbit import gather_keys_oauth2 as Oauth2

#datetime 
import datetime

import time
import json

In [2]:
def authorize_fitbit_app():
    '''
    Authorize the script to access my Fitbit data
    and return fitbit python object
    '''
    #app id
    client_id = '2287X8'
    #app password
    client_secret = '2dfcb152b035ca0bdff3d99abc6af66e'
    #connect to server for access and refresh tokens
    server = Oauth2.OAuth2Server(client_id, client_secret)
    server.browser_authorize()
    ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
    REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
    #instantiate fitbit object
    client = fitbit.Fitbit(client_id, client_secret, 
                           oauth2=True, access_token=ACCESS_TOKEN, 
                           refresh_token=REFRESH_TOKEN)
    return client

def get_weight(dates):
    '''
    Get historical weight data from Fitbit.
    Note: I log my weight using the MyFitnessPal app, which Fitbit
    downloads from. Because MyFitnessPal does not have a public API, I would
    have to download my weight statistics manually. This script allows me to
    programmatically access that data through Fitbit's connection to MyFitnessPal
    I also use this instead of the time series fitbit method because time series
    method imputes days where no weight was entered as the last known date. This way,
    I can manually interpolate the interim days' weight myself.
    '''
    #create empty list
    weight = []
    #iterate through period index of frequency month
    for date in dates:
        #create dataframe of weight data starting from the first of the month
        #to the end of the month
        df = pd.DataFrame(client.get_bodyweight(base_date = date, period = '1m')['weight'])
        #append to list
        weight.append(df)
    #concatenate into dataframe and export
    weight = pd.concat(weight, ignore_index = True, sort = False)
    weight.to_csv('weight.csv', index = False)
    #return for console inspection
    return weight



def get_sleep(days):
    sleep_list = []
    for day in days:
        json = client.get_sleep(day)
        stats = pd.DataFrame(json['sleep'])
        summary = pd.DataFrame(json['summary'], index = [0])
        df = pd.concat([stats, summary], axis = 1)
        sleep_list.append(df)
        time.sleep(2)
    sleep = pd.concat(sleep_list, ignore_index = True, sort = False)
    sleep.to_csv('sleep.csv', index = False)
    return sleep

def get_heart_rate_data(days):
    '''
    get heart rata data for every second during a day
    when I worked out
    '''
    for counter, day in enumerate(days, 1):
        day = str(day.date())
        heart_rate = client.intraday_time_series('activities/heart', 
                                                base_date= day, 
                                                detail_level='1sec')
        with open(f'heart_rate_data/heart_rate_{day}.json', 'w') as outfile:
            json.dump(heart_rate, outfile)
        if counter % 100 == 0:
            print('Sleeping')
            #Sleeping for an hour
            time.sleep(3600)


In [3]:
client = authorize_fitbit_app()

[27/Dec/2018:00:55:38] ENGINE Listening for SIGTERM.
[27/Dec/2018:00:55:38] ENGINE Bus STARTING
[27/Dec/2018:00:55:38] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[27/Dec/2018:00:55:38] ENGINE Started monitor thread 'Autoreloader'.
[27/Dec/2018:00:55:38] ENGINE Serving on http://127.0.0.1:8080
[27/Dec/2018:00:55:38] ENGINE Bus STARTED


127.0.0.1 - - [27/Dec/2018:00:55:40] "GET /?code=4a6a949f771ec797486f7b30a1863d2961d3fabd&state=YqpxJcfCQMng6POMDqVWCYEAAzuD83 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"


[27/Dec/2018:00:55:41] ENGINE Bus STOPPING
[27/Dec/2018:00:55:50] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[27/Dec/2018:00:55:50] ENGINE Removed handler for console events.
[27/Dec/2018:00:55:50] ENGINE Stopped thread 'Autoreloader'.
[27/Dec/2018:00:55:50] ENGINE Bus STOPPED
[27/Dec/2018:00:55:50] ENGINE Bus EXITING
[27/Dec/2018:00:55:50] ENGINE Waiting for child threads to terminate...
[27/Dec/2018:00:55:50] ENGINE Bus EXITED
[27/Dec/2018:00:55:50] ENGINE Waiting for thread Thread-19.


In [4]:
#get the date I joined Fitbit
date_joined_fitbit = client.user_profile_get()['user']['memberSince']
today = datetime.datetime.now().date()
dates = pd.date_range(start = date_joined_fitbit, end = today, freq = 'MS')
days = pd.date_range(start = date_joined_fitbit, end = today, freq = 'D')

In [5]:
lifts = pd.read_csv('lifts.csv', index_col = 'date')
lifting_days = pd.to_datetime(lifts.index)
intersection = lifting_days.intersection(days).unique()

In [6]:
get_heart_rate_data(intersection)

HTTPTooManyRequests: Too Many Requests